# TFT Library

This program was an implementation of the [PIC32 TFT library](https://people.ece.cornell.edu/land/courses/ece4760/PIC32/index_TFT_display.html) for the Raspberry Pi Pico. The TFT display is interfaced using SPI. However, I used PIO to create my own SPI channel running at 31.25MHz to send data to the TFT display. The PIO driver code is pretty similar to the [PIO DDS](https://parthssharma.github.io/Pico/PIODDS.html) program except for a few modifications which have been addressed in the subsequent sections. The resources for the project include the [C SDK User Guide](https://datasheets.raspberrypi.org/pico/raspberry-pi-pico-c-sdk.pdf), the [RP2040 Datasheet](https://datasheets.raspberrypi.org/rp2040/rp2040-datasheet.pdf), the ECE 4760 [course website](https://people.ece.cornell.edu/land/courses/ece4760/) and Prof. Hunter's [website](https://vha3.github.io/).

**Note: The fundamental difference between my program and the program for PIC32 is that instead of transmitting 16-bit words, I am transmitting 2 8-bit words whenever needed.**

---

## The complete code

### TFT_PIO.c

This is the driver file which is used to test the created library.

```c
#include <stdio.h> //The standard C library
#include <stdlib.h> //C stdlib
#include "pico/stdlib.h" //Standard library for Pico
#include <math.h> //The standard math library
#include "hardware/gpio.h" //The hardware GPIO library
#include "pico/time.h" //The pico time library
#include "hardware/irq.h" //The hardware interrupt library
#include "hardware/pwm.h" //The hardware PWM library
#include "hardware/pio.h" //The hardware PIO library
#include "TFTMaster.h" //The TFT Master library

int main(){ //The program running on core 0
	int i, j; //The insex variables
	unsigned short col, count = 0; //Variables to store current colour and count
    stdio_init_all(); //Initialize all of the present standard stdio types that are linked into the binary
	tft_init_hw(); //Initialize the hardware for the TFT
	tft_begin(); //Initialize the TFT
	tft_setRotation(0); //Set TFT rotation
	tft_fillScreen(ILI9340_BLACK); //Fill the entire screen with black colour
	
	while(1){ //Infinite while loop
		unsigned long begin_time = (unsigned long)(get_absolute_time() / 1000); //Get the start time
		switch(count){ //Based on the current count, switch different colours
			case 0: col = ILI9340_BLUE;
					break;
			case 1: col = ILI9340_RED;
					break;
			case 2: col = ILI9340_GREEN;
					break;
			case 3: col = ILI9340_CYAN;
					break;
			case 4: col = ILI9340_MAGENTA;
					break;
			case 5: col = ILI9340_YELLOW;
					break;
			case 6: col = ILI9340_WHITE;
					break;
		}
		for(i = 0; i < ILI9340_TFTWIDTH / 4; i++){
			for(j = 0; j < ILI9340_TFTHEIGHT / 4; j++){
				//tft_drawRect(i << 2, j << 2, 4, 4, col); //Simply drawing a rectangle takes 222 ms
				tft_fillRect(i << 2, j << 2, 4, 4, col); //Filling the entire rectangle surprisingly takes 110 ms
			}
		}
		count = (count + 1) % 7; //Increment the count and keep it between 0-6
		unsigned char exTime = 33 - ((unsigned long)(get_absolute_time() / 1000) - begin_time); //Calculate the amount of time to sleep for to achieve 30fps
		printf("%u\n", exTime); //Print the time out
	}
}

```

<br>

### SPIPIO.pio

This is the driver code for the PIO state machine and can be downloaded [here](parthssharma.github.io/Pico/Files/SPIPIO.pio).

```asm
.program spi_cpha0_cs ;Program name
.side_set 1 ;Set 1 pin for sideset

; Drive SPI
; Pin assignments:
; - SCK is side-set bit 0
; - MOSI is OUT bit 0 (host-to-device)

.wrap_target ;Free 0 cycle unconditional jump
bitloop: ;Bitloop label
public entry_point: ;The entry point for the program
    out pins, 1        side 0x0 [1] ;Output the bit on pin, sideset the clock
    jmp x-- bitloop    side 0x1 [1] ;Jump to bitloop if bit counter still available

    out pins, 1        side 0x0 ;Output the bit on pin, sideset the clock
    mov x, y           side 0x0     ;Reload bit counter from Y
    jmp !osre bitloop  side 0x1 [1] ;Fall-through if TXF empties

    irq 0              side 0x0 [1] ;Set IRQ 0 flag
.wrap

;Helper function

% c-sdk {
#include "hardware/gpio.h" //The hardware GPIO library
static inline void pio_spi_cs_init(PIO pio, uint sm, uint prog_offs, uint n_bits, int clkdiv, bool cpha, bool cpol, uint pin_sck, uint pin_mosi){ //The PIO SPI initialize functions
    pio_sm_config c = spi_cpha0_cs_program_get_default_config(prog_offs); //Get default configurations for the PIO state machine
    sm_config_set_out_pins(&c, pin_mosi, 1); //Set the 'out' pins in a state machine configuration
    sm_config_set_sideset_pins(&c, pin_sck); //Set the 'sideset' pins in a state machine configuration
    sm_config_set_out_shift(&c, false, true, n_bits); //Setup 'out' shifting parameters in a state machine configuration
    sm_config_set_clkdiv(&c, clkdiv); //Set the state machine clock divider

    pio_sm_set_pins_with_mask(pio, sm, 0, (1u << pin_sck) | (1u << pin_mosi)); //Use a state machine to set a value on multiple pins for the PIO instance
    pio_sm_set_pindirs_with_mask(pio, sm, (1u << pin_sck) | (1u << pin_mosi), (1u << pin_sck) | (1u << pin_mosi)); //Use a state machine to set the pin directions for multiple pins for the PIO instance
    pio_gpio_init(pio, pin_mosi); //Setup the function select for a GPIO to use output from the given PIO instance
    pio_gpio_init(pio, pin_sck); //Setup the function select for a GPIO to use output from the given PIO instance
    //pio_gpio_init(pio, pin_sck + 1); //Setup the function select for a GPIO to use output from the given PIO instance
    gpio_set_outover(pin_sck, cpol ? GPIO_OVERRIDE_INVERT : GPIO_OVERRIDE_NORMAL); //Set GPIO output override
	

    uint entry_point = prog_offs + spi_cpha0_cs_offset_entry_point; //The offset entry point
    pio_sm_init(pio, sm, entry_point, &c); //Resets the state machine to a consistent state, and configures it
    pio_sm_exec(pio, sm, pio_encode_set(pio_x, n_bits - 2)); //Put n_bits - 2 in pio_x
    pio_sm_exec(pio, sm, pio_encode_set(pio_y, n_bits - 2)); //Put n_bits - 2 in pio_y
    pio_sm_set_enabled(pio, sm, true); //Enable or disable a PIO state machine
}
%}

```

---

<br>

### SPIPIO.pio

This is the driver code for the PIO state machine and can be downloaded [here](parthssharma.github.io/Pico/Files/SPIPIO.pio).

```asm
.program spi_cpha0_cs ;Program name
.side_set 1 ;Set 1 pin for sideset

; Drive SPI
; Pin assignments:
; - SCK is side-set bit 0
; - MOSI is OUT bit 0 (host-to-device)

.wrap_target ;Free 0 cycle unconditional jump
bitloop: ;Bitloop label
public entry_point: ;The entry point for the program
    out pins, 1        side 0x0 [1] ;Output the bit on pin, sideset the clock
    jmp x-- bitloop    side 0x1 [1] ;Jump to bitloop if bit counter still available

    out pins, 1        side 0x0 ;Output the bit on pin, sideset the clock
    mov x, y           side 0x0     ;Reload bit counter from Y
    jmp !osre bitloop  side 0x1 [1] ;Fall-through if TXF empties

    irq 0              side 0x0 [1] ;Set IRQ 0 flag
.wrap

;Helper function

% c-sdk {
#include "hardware/gpio.h" //The hardware GPIO library
static inline void pio_spi_cs_init(PIO pio, uint sm, uint prog_offs, uint n_bits, int clkdiv, bool cpha, bool cpol, uint pin_sck, uint pin_mosi){ //The PIO SPI initialize functions
    pio_sm_config c = spi_cpha0_cs_program_get_default_config(prog_offs); //Get default configurations for the PIO state machine
    sm_config_set_out_pins(&c, pin_mosi, 1); //Set the 'out' pins in a state machine configuration
    sm_config_set_sideset_pins(&c, pin_sck); //Set the 'sideset' pins in a state machine configuration
    sm_config_set_out_shift(&c, false, true, n_bits); //Setup 'out' shifting parameters in a state machine configuration
    sm_config_set_clkdiv(&c, clkdiv); //Set the state machine clock divider

    pio_sm_set_pins_with_mask(pio, sm, 0, (1u << pin_sck) | (1u << pin_mosi)); //Use a state machine to set a value on multiple pins for the PIO instance
    pio_sm_set_pindirs_with_mask(pio, sm, (1u << pin_sck) | (1u << pin_mosi), (1u << pin_sck) | (1u << pin_mosi)); //Use a state machine to set the pin directions for multiple pins for the PIO instance
    pio_gpio_init(pio, pin_mosi); //Setup the function select for a GPIO to use output from the given PIO instance
    pio_gpio_init(pio, pin_sck); //Setup the function select for a GPIO to use output from the given PIO instance
    //pio_gpio_init(pio, pin_sck + 1); //Setup the function select for a GPIO to use output from the given PIO instance
    gpio_set_outover(pin_sck, cpol ? GPIO_OVERRIDE_INVERT : GPIO_OVERRIDE_NORMAL); //Set GPIO output override
	

    uint entry_point = prog_offs + spi_cpha0_cs_offset_entry_point; //The offset entry point
    pio_sm_init(pio, sm, entry_point, &c); //Resets the state machine to a consistent state, and configures it
    pio_sm_exec(pio, sm, pio_encode_set(pio_x, n_bits - 2)); //Put n_bits - 2 in pio_x
    pio_sm_exec(pio, sm, pio_encode_set(pio_y, n_bits - 2)); //Put n_bits - 2 in pio_y
    pio_sm_set_enabled(pio, sm, true); //Enable or disable a PIO state machine
}
%}

```

---

<br>

## The output

In order to view the output of the DAC, I used an oscilloscope. As it is quite evident from the oscilloscope output, the output of the DAC is a sine wave of the desired frequency.

<div style="display: flex; justify-content: center;">
  <img src="https://parthssharma.github.io/Pico/Files/PIOSPIOutput.jpeg" style="width: 700px; height: 446px;" >
</div>
<figure>
    <center><figcaption>Output of the DAC</figcaption></center>
</figure>

---

## CMakeLists.txt

```cmake
cmake_minimum_required(VERSION 3.13)

include(pico_sdk_import.cmake)

project(PIODDSDAC)

pico_sdk_init()

add_executable(PIODDSDAC PIODDSDAC.c)

pico_generate_pio_header(PIODDSDAC ${CMAKE_CURRENT_LIST_DIR}/PIODDS.pio)

pico_enable_stdio_usb(PIODDSDAC 1)
pico_enable_stdio_uart(PIODDSDAC 1)

pico_add_extra_outputs(PIODDSDAC)

target_link_libraries(PIODDSDAC pico_stdlib pico_time hardware_gpio hardware_irq hardware_pwm hardware_adc hardware_pio)
```

---